In [59]:
import pandas as pd
import numpy as np
import sklearn
import math as mt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

from operator import itemgetter
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds

In [60]:
def DCGp(i, p, rel):
    tot = 0
    for i in range(p):
        tot += (2**rel[i]-1)/mt.log2((i+1)+1)
    return tot

In [61]:
def IDCGp(i, rel):
    tot = 0
    for i in range(abs(rel)):
        tot += (2**rel[i]-1)/mt.log2((i+1)+1)
    return tot

In [62]:
def nDCGp(i, p, rel):
    return DCGp(i, p, rel)/IDCGp(i, rel)

In [63]:
DF = pd.read_csv("citeulike-a/raw-data.csv",encoding='ISO-8859-1')#'ISO-8859-1')

In [64]:
DF.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,..."
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...


In [65]:
DF.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...


In [66]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 5 columns):
doc.id          16980 non-null int64
title           16980 non-null object
citeulike.id    16980 non-null float64
raw.title       16980 non-null object
raw.abstract    16980 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 663.4+ KB


In [67]:
citations = pd.read_csv("citeulike-a/citations.dat", header = None, delimiter='\n')

In [68]:
citations.columns = ["citations"]

In [69]:
citations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 1 columns):
citations    16980 non-null object
dtypes: object(1)
memory usage: 132.7+ KB


In [70]:
citations.head()

,citations
0,3 2 485 3284
1,16 42 43 60 113 116 161 252 1543 1782 1947 414...
2,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
3,0
4,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...


In [71]:
citations2=citations[1:len(citations)-1].copy()

In [72]:
#citations2.index.rename('doc.id',inplace=True)

In [73]:
citations2.reset_index(inplace=True)

In [74]:
citations2.tail()

,index,citations
16973,16974,12 1631 10468 10587 11949 12665 13198 14527 14...
16974,16975,16 420 6357 6371 9466 12007 12096 12660 13307 ...
16975,16976,8 7991 15184 15944 16272 16304 16451 16619 16722
16976,16977,14 418 10406 10558 11796 14709 15093 15248 152...
16977,16978,0


In [75]:
DF2=pd.concat([DF,citations2],axis=1)
DF2

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,16 42 43 60 113 116 161 252 1543 1782 1947 414...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,0
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,37 2 15 22 23 28 254 469 477 481 483 484 485 4...
5,6,random graphs with arbitrary degree distributi...,48.0,Random graphs with arbitrary degree distributi...,Recent work on the structure of social network...,6.0,14 49 1533 3646 7689 8302 10608 11649 12757 12...
6,7,artificial gene networks for objective compari...,49.0,Artificial gene networks for objective compari...,Motivation: Large-scale gene expression profil...,7.0,1 169
7,8,the segment polarity network is a robust devel...,50.0,The segment polarity network is a robust devel...,"All insects possess homologous segments, but s...",8.0,0
8,9,the evolutionary origin of complex features,52.0,The evolutionary origin of complex features.,A long-standing challenge to evolutionary theo...,9.0,9 3738 3757 4263 4298 6043 7014 9311 10171 15763
9,10,early language acquisition cracking the speech...,60.0,Early language acquisition: cracking the speec...,"Infants learn language with remarkable speed, ...",10.0,10 2 15 477 1200 1217 3937 5058 5561 8649 8958


In [76]:
type(DF2['citations'][0])

str

In [77]:
N = 10

In [78]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,16 42 43 60 113 116 161 252 1543 1782 1947 414...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,0
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,37 2 15 22 23 28 254 469 477 481 483 484 485 4...


In [79]:
DF2.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...,16976.0,8 7991 15184 15944 16272 16304 16451 16619 16722
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...,16977.0,14 418 10406 10558 11796 14709 15093 15248 152...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...,16978.0,0
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...,NaN,NaN
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...,NaN,NaN


In [80]:
DF2.dropna(inplace=True,axis=0)

In [81]:
DF2['citations']=DF2['citations'].apply(lambda x: x.split(','))

In [82]:
del DF
del citations
del citations2

In [ ]:
IdFreq={}

In [ ]:
def docIdFreq (c_list):
    for element in c_list:
        if element in IdFreq:
            IdFreq[element] = IdFreq[element]+1
        else:
            IdFreq[element] = 1

In [ ]:
DF2['citations'].apply(lambda x: docIdFreq(x));

In [ ]:
most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)

In [ ]:
del most_popular[0]

In [ ]:
most_popular[0:N]

In [ ]:
DF2.drop('index',axis=1,inplace=True)

In [ ]:
DF2.head()

In [ ]:
DF2[DF2['doc.id']==1]['raw.title'].values[0]

# Most popular article 

'The metabolic world of Escherichia coli is not small'

In [ ]:
UR = pd.read_csv("citeulike-a/users.dat",delimiter='\n',names=['liked_articles'],header=None)

In [ ]:
UR.head()

In [ ]:
UR['liked_articles']=UR['liked_articles'].apply(lambda x: x.split(' '))

In [ ]:
UR.head()

In [ ]:
UR.info()

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)
recommends = []


for train_index, validate_index in kf.split(UR):
    
    trainDF = pd.DataFrame(UR.loc[train_index])
    validateDF = pd.DataFrame(UR.loc[validate_index])
    print()
    
    IdFreq={}
    trainDF['liked_articles'].apply(lambda x: docIdFreq(x));
    most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)
    del most_popular[0]
    most_popular = most_popular[0:N]
    mp_docId = [ element[0] for element in most_popular]
    recommends.append(DF2[DF2['doc.id'].isin(mp_docId)]['raw.title'].values)
    
    


In [ ]:
recommends

# Random

In [ ]:
d_id=np.random.randint(low=0, high=len(DF2)-1,size=N)

In [ ]:
DF2[DF2['doc.id'].isin(d_id)]['raw.title'].values

# Content-Based

In [83]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [84]:
tfidf_matrix = tfidf_vectorizer.fit_transform(DF2['raw.abstract'])
nbrs_tfidf        = NearestNeighbors(n_neighbors=N+1,metric='cosine').fit(tfidf_matrix)

In [ ]:
def get_closest_neighbors_tfidf(title):
    row                = DF2[DF2['raw.title']==title].index[0]
    distances, indices = nbrs_tfidf.kneighbors(tfidf_matrix.getrow(row))
    titles_similar      = pd.Series(indices.flatten()).map(DF2.reset_index()['raw.title'])
    result             = pd.DataFrame({'distance':distances.flatten(), 'raw.title':titles_similar})
    return result

In [ ]:
get_closest_neighbors_tfidf('What is the goal of sensory coding')['raw.title'].values[1:N+1]

In [ ]:
type(tfidf_matrix.getrow(0))

# Collaborative FiItering

In [ ]:
docId = DF2['doc.id']

In [ ]:
len(docId)

In [ ]:
for i in range(len(docId)):
    print(i)

In [ ]:
del CountVectorizer
del Counter
del IdFreq
del TfidfVectorizer
del tfidf_matrix
del nbrs_tfidf
del tfidf_vectorizer

In [ ]:
DF2.drop(['citeulike.id','title','citations'],axis=1,inplace=True)

In [ ]:
'''
M_train = np.zeros(shape=(len(UR),len(docId)))
M_test

for i, row in UR.iterrows():
    end = int(len(row['liked_articles'])*0.2)
    for j in range(end):
        M[i,int(row['liked_articles'][j])-1]=1
'''

In [ ]:
BARRIER();

In [255]:
UR = UR.iloc[0:3999]

In [256]:
UR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 1 columns):
liked_articles    3999 non-null object
dtypes: object(1)
memory usage: 31.3+ KB


### K-fold Cross Validation

In [257]:
from random import shuffle

In [258]:
def shuffleDF(UR):
    for i, row in UR.iterrows():
        x=row['liked_articles']
        shuffle(x)
        UR.loc[i,'liked_articles'] = x

In [259]:
UR.head()

,liked_articles
0,"[70, 495, 1631, 2317, 2526, 2846, 2931, 3171, ..."
1,"[38, 493, 942, 1519, 1843, 1844, 1896, 2819, 3..."
2,"[20, 517, 791, 800, 1329, 1767, 1984, 2126, 30..."
3,"[12, 706, 709, 721, 755, 756, 776, 892, 895, 2..."
4,"[12, 761, 2678, 4034, 4035, 5439, 5494, 5758, ..."


In [260]:
shuffleDF(UR)

In [261]:
UR.head()

,liked_articles
0,"[12803, 12716, 15833, 14676, 12831, 15026, 153..."
1,"[5031, 11913, 1519, 15800, 942, 2819, 1896, 86..."
2,"[1329, 4458, 791, 517, 5266, 1984, 4792, 800, ..."
3,"[755, 2588, 756, 5056, 776, 709, 12, 706, 2991..."
4,"[11877, 5494, 7798, 11904, 9046, 4034, 5758, 1..."


In [262]:
def getTest(row,fold):
    
    UR_test=[]
    fold=fold-1
    pct=0.2
    readByUser = row['liked_articles'].copy()
    end = len(readByUser)
    end = int(end - (pct*fold))
    #print('readByUser', readByUser)
    pctValue=int(end*(pct))
    #print('pctValue: ',pctValue)
    readByUser = readByUser[end-pctValue:end+1]
    #print('readByUser Test', readByUser)
    #UR_train = np.delete(readByUser, np.s_[end-pct:end+1], axis=1)
    for element in readByUser:
        UR_test.append(int(element))
    return UR_test, readByUser

In [263]:
def getTrain(UR, fold):
    
    M = np.zeros(shape=(len(UR),len(docId)+1))
    fold=fold-1
    pct=0.2

    for i, row in UR.iterrows():
        readByUser = row['liked_articles'].copy()
        end = len(readByUser)
        end = int(end - (pct*fold))
        pctValue=int(end*(pct))
        del readByUser[end-pctValue:end+1]
        for j in range(len(readByUser)):
            element=int(readByUser[j])
            if element < 16979:
                M[i,element]=1
    return M

In [264]:
DF2.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
16973,16974,concentric circles in wmap data may provide ev...,8266079.0,Concentric circles in WMAP data may provide ev...,Conformal cyclic cosmology (CCC) posits the ex...,16974.0,[12 1631 10468 10587 11949 12665 13198 14527 1...
16974,16975,analysing biological pathways in genomewide as...,8270407.0,Analysing biological pathways in genome-wide a...,Genome-wide association (GWA) studies have ty...,16975.0,[16 420 6357 6371 9466 12007 12096 12660 13307...
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...,16976.0,[8 7991 15184 15944 16272 16304 16451 16619 16...
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...,16977.0,[14 418 10406 10558 11796 14709 15093 15248 15...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...,16978.0,[0]


In [265]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,[16 42 43 60 113 116 161 252 1543 1782 1947 41...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,[85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 4...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,[0]
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,[23 2 28 488 918 1200 1351 1861 1937 2289 3079...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,[37 2 15 22 23 28 254 469 477 481 483 484 485 ...


In [266]:
N=100

In [267]:
preds_df

,1,2,3,4,5,6,7,8,9,10,...,16969,16970,16971,16972,16973,16974,16975,16976,16977,16978
0,-0.015015,0.103280,0.104222,-0.049347,0.007374,0.016122,0.037494,0.000563,0.007841,0.003111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.013749,-0.017466,0.079506,-0.011524,0.017772,0.014171,0.025483,0.013641,0.007154,-0.002891,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.004782,-0.005978,0.005524,0.007639,-0.000691,0.002627,-0.010102,0.008048,0.005093,-0.004881,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.005727,-0.010288,0.012343,-0.001890,0.001564,-0.006253,0.007109,0.002150,0.007460,-0.001592,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.021560,0.013519,0.015767,-0.003040,-0.000281,0.008125,0.000932,0.016095,0.003312,0.002334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.002662,0.000891,-0.013520,0.000772,-0.005820,0.001802,-0.004947,0.004930,-0.003724,-0.008324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.006097,0.001081,-0.004083,-0.001346,-0.001822,-0.002716,0.000692,-0.000806,-0.000513,-0.003114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.001018,0.005766,0.046728,-0.019256,-0.003184,0.009280,-0.002425,0.004352,0.003574,0.995730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-0.004983,-0.006749,-0.003738,0.004762,-0.020891,0.000864,-0.002566,0.014195,0.008529,-0.005306,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-0.007216,0.037275,-0.003265,-0.003097,0.012501,-0.001331,0.011666,-0.003874,0.020436,0.012688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [269]:
del RecList, TestList

In [270]:
whos

Variable                       Type                Data/Info
------------------------------------------------------------
A                              DataFrame               distance             <...>ses via Web se...   16762
ContentBased                   function            <function ContentBased at 0x7f04150a1f28>
CountVectorizer                type                <class 'sklearn.feature_e<...>on.text.CountVectorizer'>
Counter                        type                <class 'collections.Counter'>
DCGp                           function            <function DCGp at 0x7f041435f8c8>
DF2                            DataFrame                  doc.id            <...>n[16978 rows x 7 columns]
IDCGp                          function            <function IDCGp at 0x7f041435f2f0>
KFold                          ABCMeta             <class 'sklearn.model_selection._split.KFold'>
N                              int                 100
NearestNeighbors               ABCMeta             <class '

In [271]:
RecList={}
TestList={}

end = len(DF2)

fold=1
    
while fold < 6:
    RecList[fold]=[]
    TestList[fold]=[]
    
    UR_train = getTrain(UR, fold)#np.delete(M, np.s_[end-twpct:end+1], axis=1) # remove columns 1 and 2
    
    U, sigma, Vt = svds(UR_train, k = 500)

    sigma = np.diag(sigma)

    print(U.shape,Vt.shape)

    all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))

    preds_df = pd.DataFrame(all_user_predicted_ratings)
    preds_df.drop(0,inplace=True,axis=1)
    for user_row_number in range(200):#range(len(UR)):
        
        sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False).index.tolist()
        UR_test = getTest(UR.iloc[user_row_number],fold)
        TestList[fold].append(UR_test)
        rec = sorted_user_predictions
        RecList[fold].append(rec[0:N])
    fold=fold+1

(3999, 500) (500, 16979)
(3999, 500) (500, 16979)
(3999, 500) (500, 16979)
(3999, 500) (500, 16979)
(3999, 500) (500, 16979)


In [ ]:
'''
recommendations = (DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].
merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'doc.id',
               right_on = 'index'))

recommendations.rename(columns = {user_row_number: 'Predictions'},inplace=True)
recommendations.drop('index',axis=1,inplace=True)
recommendations.sort_values('Predictions', ascending = False,inplace=True)
recommendations.dropna(axis=0,inplace=True)
UR_test = getTest(UR.iloc[user_row_number],fold)
TestList[fold].append(UR_test)
recommendations.reset_index(inplace=True)
'''

##### Salva preds_df

In [ ]:
fold = 1
while fold < 6:
    
    UR_train = getTrain(UR, fold)#np.delete(M, np.s_[end-twpct:end+1], axis=1) # remove columns 1 and 2
    
    U, sigma, Vt = svds(UR_train, k = 500)

    sigma = np.diag(sigma)

    print(U.shape,Vt.shape)

    all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))

    preds_df = pd.DataFrame(all_user_predicted_ratings)
    preds_df.drop(0,inplace=True,axis=1)
    preds_df.to_csv('preds'+'Fold'+str(fold))
    fold=fold+1

In [139]:
N=100

In [207]:
RecList={}
TestList={}
fold=1
    

while fold < 6:

    #preds_df = pd.read_csv('preds'+'Fold'+str(fold))
    #preds_df.drop('Unnamed: 0',axis=1,inplace=True)
    RecList[fold]=[]
    TestList[fold]=[]
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(DF2['raw.abstract'])
    print('Fold: ',fold)
    for user_row_number in range(200):#range(len(UR)):
        
        #sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False).index.tolist()
        
        UR_test, readByUser = getTest(UR.iloc[user_row_number],fold)
        
        sorted_user_predictions = ContentBased(user_row_number,readByUser, tfidf_matrix)
        
        TestList[fold].append(UR_test)
        
        rec=sorted_user_predictions['doc.id'].tolist()
        
        RecList[fold].append(rec[0:N])
    int(fold)
    fold=fold+1

Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5


In [125]:
def getDocId(title):
    return DF2[DF2['raw.title']==title]['doc.id'].values[0]

In [149]:
def ContentBased2(user_number, UserList):

    tfidf_matrix = tfidf_vectorizer.transform(DF2['raw.abstract'])#[~DF2['doc.id'].isin(UserList)]['raw.abstract'])
  
    nbrs_tfidf        = NearestNeighbors(n_neighbors=N+1,metric='cosine').fit(tfidf_matrix)

    distances, indices = nbrs_tfidf.kneighbors(getTFIDFUser(user_number))
    titles_similar      = pd.Series(indices.flatten()).map(DF2.reset_index()['raw.title'])
    result             = pd.DataFrame({'distance':distances.flatten(), 'raw.title':titles_similar})
    #result['doc.id']=result['raw.title'].apply(lambda x: getDocId(x))
    return result

In [205]:
def ContentBased(user_number, UserList, tfidf_matrix):
 
    #tfidf_matrix = tfidf_vectorizer.fit_transform(DF2[''])
    nbrs_tfidf        = NearestNeighbors(n_neighbors=N+1,metric='cosine',algorithm='auto').fit(tfidf_matrix)

    distances, indices = nbrs_tfidf.kneighbors(getTFIDFUser(user_number))
    titles_similar      = pd.Series(indices.flatten()).map(DF2.reset_index()['raw.title'])
    result             = pd.DataFrame({'distance':distances.flatten(), 'raw.title':titles_similar})
    result['doc.id']=result['raw.title'].apply(lambda x: getDocId(x))
    return result[~result['doc.id'].isin(readByUser)]

In [233]:
#Valor do Prediction nao importa, so se a recomendacao esta no teste ou nao

def DCGp(N, Rel):
    tot = 0
    for i in range(len(Rel)):
        tot += (2**Rel[i])/mt.log2((i+1)+1)
    return tot

In [234]:
def IDCGp(N, Rel):
    tot = 0
    for i in range(len(Rel)):
        tot += (2**Rel[i])/mt.log2((i+1)+1)
    return tot

In [235]:
def nDCGp(N, Rel):
    
    return (DCGp(N,Rel)/IDCGp(N, Rel))

In [280]:
nDCGList={}
def nDCG (RecList, TestList):
    for i in range(len(RecList)):
        i=i+1
        nDCGList[i]=[]
        print('Fold ',i,'\n\n')
        for j in range(len(RecList[i])):
            Rel=[]
            print('\nUsuario: ',j+1)
            print('Rec:',RecList[i][j])
            print('\nTest:', TestList[i][j])
            
            for x in RecList[i][j]:
                if x in TestList[i][j][0]:
                    Rel.append(1)
                else:
                    Rel.append(0)
            
            print('\nRel:' ,Rel)
            
            #for k in range(N):
            nDCGList[i].append(nDCGp(N, Rel))
            mean=np.mean(nDCGList[i])
            print('nDCG: ',mean,'\n\n')
            del nDCGList[i][:]
            nDCGList[i].append(mean)
            #print(nDCGList[i])
        print('****************************************************************************************************')
        mean=np.mean(nDCGList[i])
        print('Fold: ',i,'nDCG mean: ',mean,'\n\n')
        del nDCGList[i][:]
        nDCGList[i].append(mean)

In [281]:
nDCG(RecList, TestList)

Fold  1 



Usuario:  1
Rec: [3591, 6874, 14676, 12805, 5991, 10288, 12716, 15894, 10008, 6103, 10508, 3297, 2317, 12684, 8903, 1631, 12803, 70, 15336, 495, 6968, 12831, 12804, 11105, 11853, 10272, 3171, 4889, 12802, 15191, 2846, 15165, 2931, 3770, 4871, 11009, 13924, 8748, 10677, 3332, 7867, 11650, 13187, 11320, 3950, 15300, 15833, 11919, 10588, 6655, 15140, 14851, 15026, 14662, 16163, 2526, 4662, 5114, 16424, 9907, 7801, 4624, 15355, 11309, 6543, 15221, 12064, 3097, 49, 9422, 4139, 3172, 2677, 9629, 45, 9402, 7171, 6135, 15946, 12578, 12619, 2676, 3404, 2648, 11230, 15282, 5936, 16089, 2946, 15330, 4061, 8128, 5944, 4622, 1120, 5763, 6356, 10287, 7066, 15492]

Test: ([12916, 3404, 13172, 5324, 5614, 3595, 10204, 5325, 15282, 7106, 13363, 11226, 4626, 13923], ['12916', '3404', '13172', '5324', '5614', '3595', '10204', '5325', '15282', '7106', '13363', '11226', '4626', '13923'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0 



Usuario:  163
Rec: [29, 2306, 3042, 4486, 3643, 4104, 6189, 5534, 3037, 4085, 3194, 3195, 818, 719, 9439, 6231, 4367, 1345, 3641, 7517, 14963, 3642, 3161, 4804, 8012, 220, 7467, 7731, 1253, 69, 1256, 5588, 2294, 7787, 5392, 12187, 10631, 1254, 2397, 8910, 9743, 7663, 6068, 4725, 151, 3767, 4633, 2399, 10556, 7735, 6030, 5989, 3644, 5356, 10765, 2616, 8832, 190, 3036, 7092, 16103, 7530, 4037, 1213, 6511, 10398, 13216, 3572, 12631, 4629, 7531, 16111, 2409, 10237, 7700, 11168, 3285, 7237, 2890, 7529, 1146, 2279, 9681, 6959, 16655, 61, 3713, 3553, 5153, 2400, 5317, 2405, 4909, 7469, 5221, 7471, 7514, 4509, 4181, 97]

Test: ([1252, 6489, 9719, 5660, 1251, 13946, 5989, 3644, 

nDCG:  1.0 



Usuario:  195
Rec: [4179, 35, 64, 169, 116, 1500, 113, 1, 6850, 2618, 2621, 159, 2826, 5599, 5160, 252, 1567, 3628, 4072, 4965, 2490, 3990, 63, 2839, 5596, 4393, 2638, 4505, 5598, 7359, 7901, 60, 5161, 4141, 1652, 1523, 1643, 3865, 5019, 5287, 8134, 5924, 201, 8906, 2813, 123, 7, 3743, 2812, 1228, 4798, 6422, 1583, 2940, 1623, 2491, 1744, 5254, 1635, 7635, 1502, 8163, 4928, 475, 1555, 4886, 5151, 1528, 4215, 1604, 4145, 5259, 299, 4927, 2814, 115, 1797, 4385, 1570, 4950, 539, 275, 472, 4799, 4124, 1977, 5441, 5695, 1609, 9148, 2724, 2016, 8996, 6418, 3974, 1499, 532, 279, 7035, 533]

Test: ([4503, 5596, 123, 4547, 5019, 3743, 4505, 25], ['4503', '5596', '123', '4547', '5019', '3743', '4505', '25'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0 



Usuario:  182
Rec: [30, 2549, 718, 2552, 11758, 11762, 2760, 11753, 11754, 6075, 9998, 1082, 1898, 2970, 521, 14267, 10555, 2550, 11755, 11760, 11752, 9654, 11756, 6083, 7558, 11759, 2008, 5721, 561, 7915, 11761, 8406, 8094, 1901, 6069, 10271, 9219, 14842, 1482, 10585, 1344, 14611, 11757, 11925, 13017, 10031, 4894, 10025, 3187, 9473, 2323, 14072, 2551, 12773, 7465, 54, 5301, 12453, 904, 10652, 14410, 4181, 3496, 9974, 2295, 14236, 2835, 7514, 8964, 11720, 9276, 3294, 2926, 8832, 13026, 12875, 7386, 74, 6068, 10217, 10765, 9295, 9729, 287, 1794, 13281, 15024, 962, 6505, 6621, 9102, 9129, 10766, 900, 8326, 3268, 15842, 628, 6768, 1142]

Test: ([11755, 1482, 8995, 13017, 13


Usuario:  87
Rec: [14607, 755, 9103, 335, 4020, 14877, 7658, 52, 12377, 2207, 4102, 5453, 10995, 676, 2769, 10214, 3810, 12670, 2467, 4807, 12270, 12271, 140, 13144, 2534, 12244, 2953, 6794, 6599, 659, 16229, 10464, 3095, 14158, 16914, 893, 8670, 348, 15190, 8426, 4511, 14576, 10184, 7174, 12547, 197, 5120, 1020, 2233, 2216, 3373, 12213, 6541, 1416, 11618, 10180, 12258, 1389, 2331, 3160, 1056, 12571, 3424, 14733, 2096, 8598, 15113, 1743, 2299, 6964, 9667, 6168, 1665, 3746, 1091, 12116, 14336, 11186, 3930, 4958, 15787, 3707, 9968, 5355, 6754, 9400, 5684, 11964, 7295, 13405, 8160, 12613, 3852, 2038, 13291, 46, 16098, 450, 15364, 6347]

Test: ([9785, 13105, 637, 14052, 4036, 6747, 7605, 12244, 7295, 13405, 724], ['9785', '13105', '637', '14052', '4036', '6747', '7605', '12244', '7295', '13405', '724'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [238]:
nDCGList

{1: [1.0], 2: [1.0], 3: [1.0], 4: [1.0], 5: [1.0]}

In [229]:
type(TestList[1][2][0])

int

In [241]:
len(set(RecList[1][0])-set(TestList[1][0]))

99

In [272]:
def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [275]:
def nDCG11 (RecList, TestList):
    for i in range(len(RecList)):
        i=i+1
        nDCGList[i]=[]
        print('Fold ',i,'\n\n')
        for j in range(len(RecList[i])):
            Rel=[]
            print('\nUsuario: ',j+1)
            print('Rec:',RecList[i][j])
            print('\nTest:', TestList[i][j])
            
            for x in RecList[i][j]:
                if x in TestList[i][j][0]:
                    Rel.append(1)
                else:
                    Rel.append(0)
            
            print('\nRel:' ,Rel)
            
            #for k in range(N):
            nDCGList[i].append(ndcg_at_k(Rel,N))
            mean=np.mean(nDCGList[i])
            print('nDCG: ',mean,'\n\n')
            del nDCGList[i][:]
            nDCGList[i].append(mean)
            #print(nDCGList[i])
        print('****************************************************************************************************')
        mean=np.mean(nDCGList[i])
        print('Fold: ',i,'nDCG mean: ',mean,'\n\n')
        del nDCGList[i][:]
        nDCGList[i].append(mean)

In [276]:
nDCG11(RecList,TestList)

Fold  1 



Usuario:  1
Rec: [3591, 6874, 14676, 12805, 5991, 10288, 12716, 15894, 10008, 6103, 10508, 3297, 2317, 12684, 8903, 1631, 12803, 70, 15336, 495, 6968, 12831, 12804, 11105, 11853, 10272, 3171, 4889, 12802, 15191, 2846, 15165, 2931, 3770, 4871, 11009, 13924, 8748, 10677, 3332, 7867, 11650, 13187, 11320, 3950, 15300, 15833, 11919, 10588, 6655, 15140, 14851, 15026, 14662, 16163, 2526, 4662, 5114, 16424, 9907, 7801, 4624, 15355, 11309, 6543, 15221, 12064, 3097, 49, 9422, 4139, 3172, 2677, 9629, 45, 9402, 7171, 6135, 15946, 12578, 12619, 2676, 3404, 2648, 11230, 15282, 5936, 16089, 2946, 15330, 4061, 8128, 5944, 4622, 1120, 5763, 6356, 10287, 7066, 15492]

Test: ([12916, 3404, 13172, 5324, 5614, 3595, 10204, 5325, 15282, 7106, 13363, 11226, 4626, 13923], ['12916', '3404', '13172', '5324', '5614', '3595', '10204', '5325', '15282', '7106', '13363', '11226', '4626', '13923'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Test: ([5880, 108, 5881, 5897, 5889, 5904, 3213, 1114, 5878, 20, 5903, 752, 5108, 1565, 5893], ['5880', '108', '5881', '5897', '5889', '5904', '3213', '1114', '5878', '20', '5903', '752', '5108', '1565', '5893'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  0.308861717208 



Usuario:  23
Rec: [1372, 3638, 1115, 5782, 286, 6971, 909, 430, 814, 3502, 7437, 2338, 6972, 2785, 2077, 10465, 8016, 3658, 9374, 9759, 5781, 5602, 4585, 1156, 7096, 4462, 399, 2787, 8112, 8178, 1358, 398, 2346, 1371, 1369, 2983, 1368, 3659, 7349, 1364, 4572, 5733, 3067, 3687, 3688, 4577, 2799, 2790, 4461, 70, 5603, 2334, 2342, 267, 2798, 7187, 2786, 7401, 7506, 6938, 6973, 5395, 3226, 2800, 775, 2797, 239, 1355, 4580, 1363, 2337, 4844, 809, 4460, 28

nDCG:  0.227181574111 



Usuario:  20
Rec: [23, 14177, 10605, 11328, 12842, 9966, 13443, 13515, 6484, 9103, 13491, 11211, 13326, 14018, 14164, 11158, 10505, 11584, 11109, 14034, 12506, 10495, 12563, 12462, 14176, 13364, 12571, 13874, 8158, 10304, 14523, 12276, 14426, 13784, 14877, 5336, 14632, 11186, 9463, 13942, 9164, 14326, 14175, 2053, 13553, 13104, 11651, 6467, 14713, 16043, 14348, 9878, 880, 9978, 12604, 13444, 12466, 8026, 14362, 8511, 11183, 12521, 11371, 13098, 15686, 8136, 477, 10430, 12213, 12679, 11016, 1117, 15897, 12205, 15094, 12844, 12467, 12689, 13099, 15071, 12469, 14309, 434, 11887, 15795, 12498, 14672, 15801, 12985, 15053, 12451, 11337, 13534, 11812, 12603, 6164, 15794, 4983, 81, 8541]

Test: ([13095, 13874, 13942, 14523, 14176], ['13095', '13874', '13942', '14523', '14176'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  0.0560903730602 



Usuario:  72
Rec: [17, 25, 1213, 490, 6110, 549, 2312, 1345, 1359, 4834, 5588, 3972, 8701, 4932, 4334, 2397, 4343, 8294, 4548, 8791, 10555, 7747, 10765, 9935, 4634, 2235, 1181, 12312, 226, 6617, 921, 6612, 3414, 709, 2404, 1365, 2323, 3839, 36, 3808, 10351, 1342, 10043, 3187, 11363, 2008, 7170, 11527, 7517, 12187, 483, 151, 2221, 13961, 6347, 10012, 39, 220, 1398, 2400, 2350, 7194, 914, 1028, 561, 13842, 6880, 7308, 2357, 13593, 243, 7625, 10481, 13463, 7915, 11783, 1448, 4020, 8158, 9000, 8569, 14521, 8756, 9939, 12744, 2009, 3767, 7746, 3120, 4725, 436, 12484, 15667, 7177, 6169, 3726, 3783, 1855, 291, 8377]

Test: ([9998, 4101, 8012, 9236, 8094, 1346], ['99

Usuario:  112
Rec: [18, 10944, 1957, 106, 6188, 28, 9578, 3932, 380, 11588, 12456, 3942, 10221, 5034, 6541, 483, 13561, 13333, 1595, 10055, 2018, 12390, 737, 10071, 8791, 7665, 10791, 1941, 7426, 16537, 15323, 12887, 2207, 1400, 6743, 8188, 1679, 4160, 1546, 3890, 468, 8326, 7429, 6495, 9710, 11156, 12296, 12425, 10907, 1888, 9100, 3941, 7025, 2, 52, 11567, 1200, 13392, 2725, 2979, 14233, 767, 247, 11589, 93, 76, 16603, 11706, 16952, 6782, 7821, 6283, 16314, 5088, 6430, 49, 8312, 14356, 103, 24, 84, 12970, 6410, 484, 14584, 5597, 12479, 16694, 2164, 531, 4130, 8994, 10484, 15249, 13479, 3007, 4556, 344, 14570, 754]

Test: ([8522, 9933, 6412, 6577], ['8522', '9933', '6412', '6577'])

Rel: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nD

In [277]:
nDCGList

{1: [0.13032334429908096],
 2: [0.18171162908359298],
 3: [0.18171162908359298],
 4: [0.18171162908359298],
 5: [0.18171162908359298]}

### Hybrid

In [201]:
def getTFIDFUser(rowNumber):
    AllString=''
    for element in UR.iloc[rowNumber]['liked_articles']:
        element=int(element)
        rawAbstract=DF2[DF2['doc.id']==element]['raw.abstract'].values[0]
        AllString=AllString+rawAbstract
        tfidf_matrix = tfidf_vectorizer.transform([AllString])

    return tfidf_matrix